# Importação dos dados do Banco de Dados "clvsol_jcafb_odoo16"

## Leitura das Credenciais

In [1]:
import pandas as pd

hosts = pd.read_csv('data/host.csv', delimiter=';', index_col='name')
def creds(index, col_name):
    return hosts.at[index, col_name]

# print(hosts)

## Parâmetros globais¶

In [2]:
import logging
import sqlite3

logging.basicConfig(level=logging.DEBUG)
_logger = logging.getLogger(__name__)


In [3]:
from xmlrpc import client

# host_name = 'tkl-odoo15-jcafb25-vm'
host_name = 'tkl-odoo16-vm-18'

server_url = creds(host_name, 'server')
db_name = creds(host_name, 'dbname')
username = creds(host_name, 'user')
password = creds(host_name, 'user_pw')
# password = creds(host_name, 'user_apikey')


## Importação dos dados de "res.country"

In [4]:
res_country_fields = ['id', 'name', 'code']

common = client.ServerProxy('%s/xmlrpc/2/common' % server_url)
user_id = common.authenticate(db_name, username, password, {})
models = client.ServerProxy('%s/xmlrpc/2/object' % server_url)

if user_id:

    search_domain = []
    res_country_objects = models.execute_kw(db_name, user_id, password,
                                            'res.country', 'search_read',
                                            [search_domain, res_country_fields],
                                            {}
                                           )
    res_country = pd.DataFrame(res_country_objects)

    # print(res_country)

    conn = sqlite3.connect('data/clvsol_jcafb_odoo16.db')

    cur = conn.cursor()
    cur.execute('DELETE FROM res_country')
    conn.commit()

    # res_country.to_sql('res_country', conn, if_exists='replace', index=False)
    res_country.to_sql('res_country', conn, if_exists='append', index=False)

    conn.close()


## Importação dos dados de "res.partner"

In [5]:
res_partner_fields = ['id', 'name', 'type',
                      'street_name','street','street_number','street_number2','street2', 'district', 'active'
                     ]

common = client.ServerProxy('%s/xmlrpc/2/common' % server_url)
user_id = common.authenticate(db_name, username, password, {})
models = client.ServerProxy('%s/xmlrpc/2/object' % server_url)

if user_id:

    search_domain = []
    res_partner_objects = models.execute_kw(db_name, user_id, password,
                                            'res.partner', 'search_read',
                                            [search_domain, res_partner_fields],
                                            {}
                                           )
    res_partner = pd.DataFrame(res_partner_objects)

    for i, row in res_partner.iterrows():

        if row['street_name'] is False:
            res_partner['street_name'].values[i] = None

        if row['street'] is False:
            res_partner['street'].values[i] = None

        if row['street_number'] is False:
            res_partner['street_number'].values[i] = None

        if row['street_number2'] is False:
            res_partner['street_number2'].values[i] = None

        if row['street2'] is False:
            res_partner['street2'].values[i] = None

        if row['district'] is False:
            res_partner['district'].values[i] = None

    # print(res_partner)

    conn = sqlite3.connect('data/clvsol_jcafb_odoo16.db')

    cur = conn.cursor()
    cur.execute('DELETE FROM res_partner')
    conn.commit()

    # res_partner.to_sql('res_partner', conn, if_exists='replace', index=False)
    res_partner.to_sql('res_partner', conn, if_exists='append', index=False)

    conn.close()
